In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings("ignore")# Ignore all warnings
from datetime import datetime, timedelta
# warnings.filterwarnings("default")

In [2]:
naming_key = pd.read_excel('../raw_data/naming_convension.xlsx',sheet_name='keys')
naming_demo = pd.read_excel('../raw_data/naming_convension.xlsx',sheet_name='Demographic')
naming_neuro = pd.read_excel('../raw_data/naming_convension.xlsx',sheet_name='neuropsychological')
naming = pd.concat([naming_key, naming_demo, naming_neuro], ignore_index=True)

## some help function

In [3]:
def calculate_visit_statistics(df):
    visit_counts = df.groupby('ID').size()
    visit_stats = {
        'average_visits': visit_counts.mean(),
        'min_visits': visit_counts.min(),
        'max_visits': visit_counts.max()
    }
    return visit_stats

In [4]:
def summarize_classification_progression(df):
    progression_dict = df.groupby('ID')['Classification'].apply(
        lambda x: [v for i, v in enumerate(x) if i == 0 or v != x.iloc[i-1]]
    ).to_dict()

    summary_map = {}
    for id, progression in progression_dict.items():
        progression_tuple = tuple(progression)
        if progression_tuple not in summary_map:
            summary_map[progression_tuple] = []
        summary_map[progression_tuple].append(id)

    for pattern, ids in summary_map.items():
        print(f"Progression Pattern: {pattern} -> RIDs: {ids}")
        print("with", len(ids), 'participants')

    return

In [5]:
def impute_demographic_data(df):
    df["Demographic.ApoE genotype"] = df.groupby("AIBL ID")["Demographic.ApoE genotype"].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

    education_mapping = {
        '0-6': (4, 6),
        '7-8': (7, 8),
        '9-12': (9, 12),
        '13-15': (13, 15),
        '15+': (16, 22)
    }

    def impute_years_of_education_exact(row):
        if pd.isna(row['Demographic.Years of Education Exact']) and row['Demographic.Years of Education'] in education_mapping:
            low, high = education_mapping[row['Demographic.Years of Education']]
            return np.random.default_rng(42).integers(low, high + 1) 
        return row['Demographic.Years of Education Exact']

    # Apply imputation function to each row
    df['Demographic.Years of Education Exact'] = df.apply(impute_years_of_education_exact, axis=1)

    return df

In [6]:
def generate_visit_column(df):
    # Sort values by ID and age
    df.sort_values(by=['ID', 'age'], inplace=True)

    # Calculate VISIT column
    def calculate_visit(group):
        first_age = group['age'].iloc[0]  # Age at first visit
        group['VISIT'] = ((group['age'] - first_age) * 12).astype(int)  # Difference in months as integer
        return group

    df = df.groupby('ID').apply(calculate_visit)

    # Resolve duplicate VISIT records
    def resolve_duplicates(group):
        # Add the index as a temporary column
        group['missing_count'] = group.isnull().sum(axis=1)
        group = group.sort_values(
            by=['VISIT', 'missing_count', 'age'],
            ascending=[True, True, False]
        )
        group = group.drop_duplicates(subset=['VISIT'], keep='first')
        group = group.drop(columns=['missing_count'])  # Clean up temporary columns
        return group

    df = df.groupby('ID').apply(resolve_duplicates)

    return df.reset_index(drop=True)



## AIBL

In [7]:
AIBL_raw = pd.read_excel('../raw_data/AIBL_2023.xlsx')
AIBL_raw['Demographic.YearMonthOfBirth']=AIBL_raw['Demographic.YearMonthOfBirth'].astype(str) + '01'
AIBL_raw['Demographic.YearMonthOfBirth'] = pd.to_datetime(AIBL_raw['Demographic.YearMonthOfBirth'], format='%Y%m%d')
AIBL_raw['Progress Summary.Date of NP assessment'] = pd.to_datetime(AIBL_raw['Progress Summary.Date of NP assessment'], format='%d/%m/%Y')
AIBL_raw['Age'] = round((AIBL_raw['Progress Summary.Date of NP assessment'] - AIBL_raw['Demographic.YearMonthOfBirth']).dt.days / 365, 2)
AIBL_raw = impute_demographic_data(AIBL_raw) ## only impute for education given the range but no exact value


In [8]:
AIBL_selected = AIBL_raw[naming['AIBL'].dropna().tolist()]
AIBL_selected.replace('NI', np.nan, inplace=True)
AIBL_selected = AIBL_selected.dropna(subset=['Age']).reset_index(drop=True)
AIBL_rename_dict = dict(zip(naming['AIBL'], naming['New_Name']))
AIBL_selected = AIBL_selected.rename(columns=AIBL_rename_dict)
AIBL_selected = generate_visit_column(AIBL_selected)

## NACC

In [9]:
def add_age_column_NACC(df):
    df['VISITDATE'] = pd.to_datetime(
        dict(
            year=df['VISITYR'], 
            month=df['VISITMO'], 
            day=df['VISITDAY']
        )
    )
    df['BIRTHDATE'] = pd.to_datetime(
        dict(
            year=df['BIRTHYR'], 
            month=df['BIRTHMO'], 
            day=1
        )
    )
    df['AGE'] = (
        (df['VISITDATE'] - df['BIRTHDATE']).dt.total_seconds() 
        / (365.25 * 24 * 60 * 60)
    ).round(2)
    if 'NACCAGE' in df.columns:
        df['AGE'].fillna(df['NACCAGE'], inplace=True)
    return df


In [10]:
NACC_raw = pd.read_csv('../raw_data/investigator_ftldlbd_nacc66.csv',index_col=False)
NACC_raw = NACC_raw[['NACCID']].join(NACC_raw.select_dtypes(include=['number']))
NACC_raw.replace([-4, -4.4, 8, 9, 88, 99, 97, 98, 96, 95, 888, 999, 8888, 9999, 88.8, 888.8, 99.9, 777], np.nan, inplace=True)
NACC_raw = add_age_column_NACC(NACC_raw)
NACC_selected = NACC_raw[naming['NACC'].dropna().tolist()]
NACC_selected.replace('NI', np.nan, inplace=True)
NACC_selected = NACC_selected.dropna(subset=['AGE']).reset_index(drop=True)

In [11]:
## check for the recalculation of age value
age_difference = abs(NACC_raw['AGE'] - NACC_raw['NACCAGE'])
difference_check = age_difference > 1
rows_with_difference = NACC_raw[difference_check]
print(f"Number of rows with a difference > 1 year: {difference_check.sum()}")
rows_with_difference

Number of rows with a difference > 1 year: 0


,NACCID,NACCADC,FORMVER,VISITMO,VISITDAY,VISITYR,NACCVNUM,NACCAVST,NACCNVST,NACCDAYS,...,LBOTHAVL,LBOCGAIT,LBOCGMO,LBOCGYR,LBOCGDY,LBOCGPOS,LBOCGAVL,VISITDATE,BIRTHDATE,AGE


In [12]:
NACC_rename_dict = dict(zip(naming['NACC'], naming['New_Name']))
NACC_selected = NACC_selected.rename(columns=NACC_rename_dict)
NACC_selected = generate_visit_column(NACC_selected)
NACC_selected.head(4)

,ID,Classification,age,sex,edu,APOE,BMI,blood_pressure_systolic,blood_pressure_diastolic,Heart_rate,...,NPIQ_Appetite_Sev,GDS,MMSE,memory_recall1,memory_recall2,Digit_Span_F,Digit_Span_B,animal,BNT,VISIT
0,NACC000011,3,62.21,2,16.0,1.0,22.7,115.0,70.0,76.0,...,NaN,0.0,30.0,12.0,11.0,NaN,6.0,20.0,27.0,0
1,NACC000011,3,63.38,2,16.0,1.0,22.5,118.0,76.0,57.0,...,NaN,1.0,29.0,13.0,NaN,7.0,6.0,21.0,28.0,14
2,NACC000011,3,64.34,2,16.0,1.0,22.1,110.0,70.0,62.0,...,NaN,0.0,30.0,12.0,NaN,NaN,6.0,21.0,28.0,25
3,NACC000011,2,65.00,2,16.0,1.0,21.7,135.0,80.0,76.0,...,NaN,0.0,28.0,11.0,10.0,7.0,NaN,19.0,28.0,33


## Check

check between AIBL and NACC, make sure with the same data structure

In [13]:
NACC_selected['ID'] = NACC_selected['ID'].str.replace('NACC', '').astype(int)
NACC_selected['APOE'] = NACC_selected['APOE'].map({1:'E3/E3' , 2:'E3/E4', 3:'E3/E2', 4:'E4/E4', 5:'E3/E2', 6:'E2/E2'})
NACC_selected['APOE4'] = NACC_selected['APOE'].apply(lambda x: 1 if isinstance(x, str) and 'E4' in x.split('/') else (0 if isinstance(x, str) else None))
# NACC_selected['height'] = NACC_selected['height'] * 2.54 # inches to cm
# NACC_selected['weight'] = NACC_selected['weight'] * 0.453592 # lbs to kg
NACC_selected['Digit_Span_F'] = (NACC_selected['Digit_Span_F'] / 12) * 16 # convert the range from 0-12 to 0-16 be consistent (later in modelling will do normalisation)
NACC_selected['Digit_Span_B'] = (NACC_selected['Digit_Span_F'] / 12) * 14 # convert the range from 0-12 to 0-14 be consistent (later in modelling will do normalisation)

NACC_selected['Classification'] = NACC_selected['Classification'].map({1:'HC' , 3:'MCI', 4:'AD'})
NACC_selected = NACC_selected[NACC_selected['Classification'].isin(['HC', 'MCI', 'AD'])]
NACC_selected.head(4)

,ID,Classification,age,sex,edu,APOE,BMI,blood_pressure_systolic,blood_pressure_diastolic,Heart_rate,...,GDS,MMSE,memory_recall1,memory_recall2,Digit_Span_F,Digit_Span_B,animal,BNT,VISIT,APOE4
0,11,MCI,62.21,2,16.0,E3/E3,22.7,115.0,70.0,76.0,...,0.0,30.0,12.0,11.0,NaN,NaN,20.0,27.0,0,0.0
1,11,MCI,63.38,2,16.0,E3/E3,22.5,118.0,76.0,57.0,...,1.0,29.0,13.0,NaN,9.333333,10.888889,21.0,28.0,14,0.0
2,11,MCI,64.34,2,16.0,E3/E3,22.1,110.0,70.0,62.0,...,0.0,30.0,12.0,NaN,NaN,NaN,21.0,28.0,25,0.0
4,34,MCI,79.00,2,15.0,E4/E4,23.0,169.0,76.0,73.0,...,0.0,NaN,NaN,NaN,NaN,NaN,15.0,NaN,0,1.0


In [14]:
AIBL_selected['sex'] = AIBL_selected['sex'].map({'Female': 2, 'Male': 1})
AIBL_selected['APOE4'] = AIBL_selected['APOE'].apply(lambda x: 1 if isinstance(x, str) and 'E4' in x.split('/') else (0 if isinstance(x, str) else None))

NPIQ_cols_convert_AIBL = ['NPIQ_Delusions', 'NPIQ_Hallucinations', 'NPIQ_Agitation', 'NPIQ_Depression', 'NPIQ_Anxiety', 'NPIQ_Elation', 'NPIQ_Apathy', 'NPIQ_Disinhibition','NPIQ_Irritability', 'NPIQ_Motor', 'NPIQ_Nighttime', 'NPIQ_Appetite']
AIBL_selected[NPIQ_cols_convert_AIBL] = AIBL_selected[NPIQ_cols_convert_AIBL].replace({'Yes': 1, 'No': 0})
NPIQ_cols_numeric_AIBL = ['NPIQ_Hallucinations_Sev', 'NPIQ_Depression_Sev', 'NPIQ_Anxiety_Sev', 'NPIQ_Elation_Sev', 'NPIQ_Apathy_Sev', 'NPIQ_Disinhibition_Sev', 'NPIQ_Irritability_Sev', 'NPIQ_Motor_Sev', 'NPIQ_Nighttime_Sev', 'NPIQ_Appetite_Sev']  
AIBL_selected[NPIQ_cols_numeric_AIBL] = AIBL_selected[NPIQ_cols_numeric_AIBL].apply(pd.to_numeric, errors='coerce')

AIBL_selected['Classification'] = AIBL_selected['Classification'].replace('MCIX', 'MCI')
AIBL_selected = AIBL_selected[AIBL_selected['Classification'].isin(['HC', 'MCI', 'AD'])] # Keep only rows where the 'classification' column contains 'HC', 'MCI', or 'AD'
AIBL_selected.head(4)

,ID,Classification,age,sex,edu,APOE,BMI,blood_pressure_systolic,blood_pressure_diastolic,Heart_rate,...,GDS,MMSE,memory_recall1,memory_recall2,Digit_Span_F,Digit_Span_B,animal,BNT,VISIT,APOE4
0,2,HC,65.30,1,16.0,E3/E3,24.82,130.0,80.0,66.0,...,2.0,29.0,15.0,14.0,9.0,5.0,40.0,NaN,0,0.0
1,2,HC,67.17,1,16.0,E3/E3,24.40,135.0,85.0,64.0,...,NaN,30.0,9.0,8.0,12.0,10.0,51.0,30.0,22,0.0
2,2,HC,68.69,1,16.0,E3/E3,23.40,121.0,69.0,62.0,...,0.0,30.0,20.0,18.0,13.0,10.0,42.0,30.0,40,0.0
3,2,HC,70.17,1,16.0,E3/E3,24.66,142.0,77.0,57.0,...,1.0,30.0,16.0,14.0,13.0,9.0,46.0,30.0,58,0.0


## Filter 

In [15]:
def check_classification(group_df):
    group_df = group_df.sort_values(by='age')
    first_non_missing = group_df['Classification'].first_valid_index()
    last_non_nan = group_df['Classification'].last_valid_index()
    ad_detected = False
    for idx, row in group_df.iterrows():
        if row['Classification'] == 'AD':
            ad_detected = True
        elif ad_detected and row['Classification'] != 'AD':
            group_df = group_df.loc[:idx - 1]
            break
    return group_df

def drop_record_num(df, num=1):
    id_counts = df['ID'].value_counts()
    valid_ids = id_counts[id_counts > num].index
    return df[df['ID'].isin(valid_ids)].reset_index(drop=True)

def drop_missing_records(df, threshold=0.6):
    return df[df.isnull().mean(axis=1) <= threshold]

In [16]:
def remove_after_first_ad(df):
    def truncate_after_ad(group):
        ad_indices = group[group['Classification'] == 'AD'].index
        if not ad_indices.empty:
            return group.loc[:ad_indices[0]]
        return group

    df = df.sort_values(by=['ID', 'age']).groupby('ID', group_keys=False).apply(truncate_after_ad).reset_index(drop=True)
    return df

In [17]:
print("the shape of DF", NACC_selected.shape, "There are ", len(NACC_selected["ID"].unique()), "unique participants")
NACC_filter = NACC_selected.sort_values(by=['ID', 'VISIT'])
NACC_filter = NACC_filter[NACC_filter.columns[~NACC_filter.columns.str.startswith('NPIQ')]]

NACC_filter = drop_missing_records(NACC_filter, threshold=0)
participants_to_remove = NACC_filter.groupby('ID').first().query("Classification == 'AD'").index
NACC_filter = NACC_filter[~NACC_filter['ID'].isin(participants_to_remove)]
NACC_filter = NACC_filter.groupby('ID').apply(check_classification).reset_index(drop=True)
NACC_filter = remove_after_first_ad(NACC_filter)
NACC_filter = drop_record_num(NACC_filter, num=3)
print("the shape of DF", NACC_filter.shape, "There are ", len(NACC_filter["ID"].unique()), "unique participants")

the shape of DF (180469, 43) There are  50678 unique participants
the shape of DF (11053, 21) There are  2073 unique participants


In [18]:
print("the shape of DF", AIBL_selected.shape, "There are ", len(AIBL_selected["ID"].unique()), "unique participants")
AIBL_filter = AIBL_selected.sort_values(by=['ID', 'VISIT'])
AIBL_filter = AIBL_filter[AIBL_filter.columns[~AIBL_filter.columns.str.startswith('NPIQ')]]
AIBL_filter = drop_missing_records(AIBL_filter, threshold=0.3)
participants_to_remove = AIBL_filter.groupby('ID').first().query("Classification == 'AD'").index
AIBL_filter = AIBL_filter[~AIBL_filter['ID'].isin(participants_to_remove)]
AIBL_filter = AIBL_filter.groupby('ID').apply(check_classification).reset_index(drop=True)
AIBL_filter = remove_after_first_ad(AIBL_filter)
AIBL_filter = drop_record_num(AIBL_filter,num=3)
print("the shape of DF", AIBL_filter.shape, "There are ", len(AIBL_filter["ID"].unique()), "unique participants")

the shape of DF (9476, 43) There are  2505 unique participants
the shape of DF (6317, 21) There are  944 unique participants


In [19]:
calculate_visit_statistics(AIBL_filter)

{'average_visits': 6.691737288135593, 'min_visits': 4, 'max_visits': 11}

In [20]:
(AIBL_filter.isna().mean() * 100).sort_values(ascending=False)

GDS                         12.220991
BMI                          7.883489
Heart_rate                   7.345259
blood_pressure_diastolic     7.139465
blood_pressure_systolic      6.917841
Digit_Span_B                 0.870666
Digit_Span_F                 0.870666
animal                       0.664873
BNT                          0.554060
memory_recall2               0.237454
edu                          0.174133
APOE4                        0.142473
APOE                         0.142473
CDR_SB                       0.126642
memory_recall1               0.110812
MMSE                         0.094982
Classification               0.000000
sex                          0.000000
age                          0.000000
VISIT                        0.000000
ID                           0.000000
dtype: float64

In [21]:
NACC_filter.to_csv('../preprocessed_data/NACC_selected.csv', index=False)
AIBL_filter.to_csv('../preprocessed_data/AIBL_selected.csv', index=False)

In [22]:
calculate_visit_statistics(AIBL_filter)

{'average_visits': 6.691737288135593, 'min_visits': 4, 'max_visits': 11}

In [23]:
calculate_visit_statistics(NACC_filter)

{'average_visits': 5.331886155330439, 'min_visits': 4, 'max_visits': 14}

In [24]:
AIBL_filter

,ID,Classification,age,sex,edu,APOE,BMI,blood_pressure_systolic,blood_pressure_diastolic,Heart_rate,...,GDS,MMSE,memory_recall1,memory_recall2,Digit_Span_F,Digit_Span_B,animal,BNT,VISIT,APOE4
0,2,HC,65.30,1,16.0,E3/E3,24.82,130.0,80.0,66.0,...,2.0,29.0,15.0,14.0,9.0,5.0,40.0,NaN,0,0.0
1,2,HC,67.17,1,16.0,E3/E3,24.40,135.0,85.0,64.0,...,NaN,30.0,9.0,8.0,12.0,10.0,51.0,30.0,22,0.0
2,2,HC,68.69,1,16.0,E3/E3,23.40,121.0,69.0,62.0,...,0.0,30.0,20.0,18.0,13.0,10.0,42.0,30.0,40,0.0
3,2,HC,70.17,1,16.0,E3/E3,24.66,142.0,77.0,57.0,...,1.0,30.0,16.0,14.0,13.0,9.0,46.0,30.0,58,0.0
4,2,HC,71.74,1,16.0,E3/E3,25.09,150.0,72.0,54.0,...,1.0,30.0,14.0,15.0,11.0,10.0,41.0,30.0,77,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6312,2635,HC,74.00,1,12.0,E3/E2,23.35,138.0,90.0,74.0,...,3.0,30.0,17.0,17.0,11.0,7.0,50.0,29.0,39,0.0
6313,2671,MCI,75.47,1,12.0,E3/E3,27.05,153.0,92.0,86.0,...,0.0,26.0,13.0,12.0,9.0,9.0,33.0,26.0,0,0.0
6314,2671,MCI,76.67,1,12.0,E3/E3,25.50,145.0,87.0,70.0,...,0.0,26.0,14.0,16.0,14.0,12.0,35.0,27.0,14,0.0
6315,2671,MCI,77.63,1,12.0,E3/E3,26.47,128.0,73.0,64.0,...,3.0,26.0,14.0,13.0,12.0,7.0,35.0,25.0,25,0.0


In [25]:
AIBL_filter[AIBL_filter['VISIT'] == 0]

,ID,Classification,age,sex,edu,APOE,BMI,blood_pressure_systolic,blood_pressure_diastolic,Heart_rate,...,GDS,MMSE,memory_recall1,memory_recall2,Digit_Span_F,Digit_Span_B,animal,BNT,VISIT,APOE4
0,2,HC,65.30,1,16.0,E3/E3,24.82,130.0,80.0,66.0,...,2.0,29.0,15.0,14.0,9.0,5.0,40.0,NaN,0,0.0
7,4,HC,84.73,2,9.0,E3/E3,NaN,NaN,NaN,NaN,...,0.0,27.0,9.0,2.0,10.0,5.0,34.0,NaN,0,0.0
12,5,MCI,73.23,1,8.0,E3/E3,34.20,130.0,90.0,64.0,...,NaN,30.0,10.0,0.0,8.0,5.0,NaN,NaN,0,0.0
16,6,HC,83.91,1,17.0,E3/E3,NaN,NaN,NaN,NaN,...,2.0,29.0,14.0,14.0,10.0,9.0,33.0,30.0,0,0.0
21,11,HC,89.56,1,12.0,E3/E3,20.40,130.0,85.0,72.0,...,NaN,29.0,9.0,7.0,10.0,5.0,34.0,NaN,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,2576,HC,69.14,1,9.0,E3/E3,23.08,147.0,81.0,67.0,...,0.0,30.0,12.0,13.0,10.0,6.0,42.0,30.0,0,0.0
6301,2600,MCI,67.54,2,17.0,E3/E3,29.56,168.0,78.0,88.0,...,6.0,27.0,13.0,13.0,10.0,2.0,46.0,28.0,0,0.0
6305,2615,MCI,71.66,1,10.0,E3/E2,30.04,148.0,88.0,45.0,...,1.0,22.0,9.0,10.0,10.0,5.0,26.0,28.0,0,0.0
6309,2635,HC,70.68,1,12.0,E3/E2,22.28,136.0,84.0,72.0,...,2.0,28.0,13.0,13.0,8.0,6.0,40.0,29.0,0,0.0


In [26]:
AIBL_filter['sex'].unique()

array([1, 2])

In [27]:
def calculate_stats(df, dataset_name):
    df = df.sort_values(by=['ID', 'VISIT'])
    baseline = df.groupby('ID').first().reset_index()

    age_mean = baseline['age'].mean()
    age_std = baseline['age'].std()

    gender_female_count = (baseline['sex'] == 2).sum()
    gender_female_percentage = (gender_female_count / len(baseline)) * 100

    education_mean = baseline['edu'].mean()
    education_std = baseline['edu'].std()

    APOE4_positive_count = (baseline['APOE4'] == 1).sum()
    APOE4_positive_percentage = (APOE4_positive_count / len(baseline)) * 100

    num_visits = df.groupby('ID')['VISIT'].count()
    num_visits_mean = num_visits.mean()
    num_visits_std = num_visits.std()

    follow_up_intervals = df.groupby('ID')['VISIT'].agg(['min', 'max'])
    follow_up_range = f"{follow_up_intervals['max'].min()} - {follow_up_intervals['max'].max()} months"

    df['AD_converter'] = df['Classification'].eq('AD').astype(int)
    AD_converters = df.groupby('ID')['AD_converter'].max().reset_index()
    AD_conversion_count = AD_converters['AD_converter'].sum()
    AD_conversion_percentage = (AD_conversion_count / AD_converters.shape[0]) * 100

    converters = df[df['ID'].isin(AD_converters[AD_converters['AD_converter'] == 1]['ID'])]
    if not converters.empty:
        time_to_conversion = converters[converters['Classification'] == 'AD'].groupby('ID')['VISIT'].min()
        time_to_conversion_mean = time_to_conversion.mean()
        time_to_conversion_std = time_to_conversion.std()
    else:
        time_to_conversion_mean = None
        time_to_conversion_std = None
    return {
        'Dataset': dataset_name,
        'Age at Baseline': f"{age_mean:.1f} ± {age_std:.1f}",
        'Gender (Female, n, %)': f"{gender_female_count} ({gender_female_percentage:.1f}%)",
        'Education (years, mean ± std)': f"{education_mean:.1f} ± {education_std:.1f}",
        'APOE4 Positive (n, %)': f"{APOE4_positive_count} ({APOE4_positive_percentage:.1f}%)",
        'Visits (mean ± std)': f"{num_visits_mean:.1f} ± {num_visits_std:.1f}",
        'Follow-up Interval': follow_up_range,
        'AD Conversion (n, %)': f"{AD_conversion_count} ({AD_conversion_percentage:.1f}%)",
        'Time to AD Conversion (mean ± std)': f"{time_to_conversion_mean:.1f} ± {time_to_conversion_std:.1f}" if time_to_conversion_mean else "N/A"
    }

datasets = [(NACC_filter, "NACC"), (AIBL_filter, "AIBL")]
summary_stats = pd.DataFrame()
for cur_df, cur_df_name in datasets:
    summary_stats = pd.concat([summary_stats, pd.DataFrame([calculate_stats(cur_df, cur_df_name)])], ignore_index=True)
summary_stats


,Dataset,Age at Baseline,"Gender (Female, n, %)","Education (years, mean ± std)","APOE4 Positive (n, %)",Visits (mean ± std),Follow-up Interval,"AD Conversion (n, %)",Time to AD Conversion (mean ± std)
0,NACC,71.9 ± 8.8,1321 (63.7%),16.0 ± 2.8,590 (28.5%),5.3 ± 1.6,25 - 202 months,100 (4.8%),77.3 ± 30.4
1,AIBL,70.5 ± 6.6,518 (54.9%),12.6 ± 3.2,247 (26.2%),6.7 ± 2.3,36 - 189 months,40 (4.2%),103.2 ± 37.2


## prepare for later survival analysis

In [28]:
def prepare_for_survival_analysis(df):
    df = df.sort_values(by=['ID', 'VISIT']).copy()
    
    
    def process_group(group):
        group['start'] = group['VISIT']
        group['stop'] = group['VISIT'].shift(-1)
        group['event'] = 0
        ad_indices = group[group['Classification'] == 'AD'].index.tolist()
        if ad_indices:
            first_ad_idx = ad_indices[0]
            # last_ad_idx = ad_indices[-1]
            first_ad_stop = group.loc[first_ad_idx, 'VISIT']
            retained = group.loc[:first_ad_idx - 1].copy()
            if not retained.empty:
                retained.iloc[-1, retained.columns.get_loc('event')] = 1
                retained.iloc[-1, retained.columns.get_loc('stop')] = first_ad_stop
            return retained
        else:
            group = group.dropna(subset=['stop'])
            # group = group[group['stop'] != 0]
            # No 'AD' occurrence; retain all records with 'event' as 0
            return group.copy()
    
    # Apply the processing to each group
    df_processed = df.groupby('ID').apply(process_group).reset_index(drop=True)
    df_processed = df_processed.drop(columns=['VISIT'])
    return df_processed

def get_participants_with_first_classification_HC(df):
    df = df.sort_values(by=['ID', 'age']) 

    # Filter participants whose first record has Classification == HC
    def filter_first_classification_HC(group):
        if group.iloc[0]['Classification'] == 'HC':
            return group
        return pd.DataFrame()

    df = df.groupby('ID').apply(filter_first_classification_HC).reset_index(drop=True)
    return df

def get_participants_with_first_classification_MCI(df):
    df = df.sort_values(by=['ID', 'age'])  # Sort by ID and VISIT

    # Filter participants whose first record has Classification == HC
    def filter_first_classification_MCI(group):
        if group.iloc[0]['Classification'] == 'MCI':
            return group
        return pd.DataFrame()

    df = df.groupby('ID').apply(filter_first_classification_MCI).reset_index(drop=True)
    return df

In [29]:
NACC_survival = prepare_for_survival_analysis(NACC_filter)
NACC_survival = drop_record_num(NACC_survival)
print("There are", NACC_survival['ID'].nunique(), "unique participants selected, with ", round((NACC_survival['event'].sum()/NACC_survival['ID'].nunique())*100, 2), "% convert to AD in NACC")


AIBL_survival = prepare_for_survival_analysis(AIBL_filter)
AIBL_survival= drop_record_num(AIBL_survival)
print("There are", AIBL_survival['ID'].nunique(), "unique participants selected, with ", round((AIBL_survival['event'].sum()/AIBL_survival['ID'].nunique())*100, 2), "% convert to AD in AIBL")

There are 2073 unique participants selected, with  4.82 % convert to AD in NACC
There are 944 unique participants selected, with  4.24 % convert to AD in AIBL


In [30]:
AIBL_survival.to_csv('../preprocessed_data/longitudinal_AIBL_full.csv', index=False)
NACC_survival.to_csv('../preprocessed_data/longitudinal_NACC_full.csv', index=False)